# Burlington VT Isochrone Generation

**Goal**: Generate walking isochrones for Burlington restaurants using:
- Overture Places (via DuckDB, direct from cloud)
- OSMnx for street network + isochrone computation
- Lonboard for visualization

In [ ]:
import duckdb
import geopandas as gpd
import osmnx as ox
import networkx as nx
from shapely.geometry import Point
import pandas as pd
import numpy as np

## 1. Get Burlington VT Boundary from Overture Divisions

Query the division_area to get bbox and WKT geometry for Burlington.

In [2]:
# Initialize DuckDB with extensions
con = duckdb.connect()
con.sql("""
    INSTALL spatial; LOAD spatial;
    INSTALL httpfs; LOAD httpfs;
    SET s3_region='us-west-2';
""")

In [3]:
# Get Burlington VT boundary from division_area
# First let's see what divisions are available for Vermont
con.sql("""
SELECT 
    names.primary as name,
    subtype,
    country
FROM read_parquet('s3://overturemaps-us-west-2/release/2026-01-21.0/theme=divisions/type=division_area/*.parquet')
WHERE 
    country = 'US'
    AND lower(names.primary) LIKE '%burlington%'
    AND region = 'US-VT'
""").df()

,name,subtype,country
0,South Burlington,locality,US
1,Burlington,locality,US


In [ ]:
# Get bbox and WKT geometry for Burlington (NOT South Burlington)
bbox, wkt_geom = con.sql("""
SELECT
    bbox,
    ST_AsText(geometry) AS wkt
FROM read_parquet('s3://overturemaps-us-west-2/release/2026-01-21.0/theme=divisions/type=division_area/*.parquet')
WHERE 
    country = 'US'
    AND region = 'US-VT'
    AND names.primary = 'Burlington'
    AND subtype = 'locality'
""").fetchall()[0]

print(f"bbox: {bbox}")
print(f"WKT length: {len(wkt_geom)} chars")

In [5]:
# Query Overture Places - first let's see what categories we get
sample = con.sql(f"""
SELECT 
    id,
    names.primary as name,
    categories,
    geometry
FROM read_parquet('s3://overturemaps-us-west-2/release/2026-01-21.0/theme=places/type=place/*.parquet')
WHERE 
    bbox.xmin <= {bbox['xmax']}
    AND bbox.xmax >= {bbox['xmin']}
    AND bbox.ymin <= {bbox['ymax']}
    AND bbox.ymax >= {bbox['ymin']}
    AND ST_Intersects(geometry, ST_GeomFromText('{wkt_geom}'))
LIMIT 10
""").df()

print(f"Sample rows: {len(sample)}")
sample

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Sample rows: 10


,id,name,categories,geometry
0,2e3131f2-ae6e-4341-a460-5b3a49dbdf32,Shelburne Bay. Lake Champlain Vermont,"{'primary': 'lake', 'alternate': ['active_life...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,588d0ff4-7b9d-4055-bace-29bd5e39d982,"Bartlett Bay, South Burlington","{'primary': 'lake', 'alternate': ['structure_a...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,8fcc7030-2ba0-4d5b-a215-1ec5f3c826c4,Executive Physical Therapy,"{'primary': 'physical_therapy', 'alternate': [...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,ba7a5f7e-6a4e-44e9-b27e-d7985acc2027,Key Collision of South Burlington,"{'primary': 'auto_body_shop', 'alternate': ['a...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,f84a00bf-8338-46cf-beec-cd4f09e77f8c,Shearer Chevrolet Buick GMC Cadillac,"{'primary': 'automotive', 'alternate': None}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
5,115035ed-3c42-4a1b-901d-c4ad8ef54fd6,W.B. Mason,"{'primary': 'office_equipment', 'alternate': [...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
6,34355064-c8ce-4b5a-8d50-9c9ce3f98b0c,Burlington Self Storage,"{'primary': 'self_storage_facility', 'alternat...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
7,5d6913b2-d279-4a35-bbde-72c607f739e1,LCB Design Group,"{'primary': 'interior_design', 'alternate': ['...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
8,d54e4f7e-7561-414a-9a79-56e9af9b0617,Portrait Gallery,"{'primary': 'professional_services', 'alternat...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
9,1c937501-b61e-4c7b-b05f-29ac270180b4,Our House Preschool,"{'primary': 'preschool', 'alternate': None}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [6]:
# Inspect the categories structure
print("Categories column type:", type(sample['categories'].iloc[0]))
print("\nSample categories:")
for i, row in sample.iterrows():
    print(f"  {row['name']}: {row['categories']}")

Categories column type: <class 'dict'>

Sample categories:
  Shelburne Bay. Lake Champlain Vermont: {'primary': 'lake', 'alternate': ['active_life', 'health_and_medical']}
  Bartlett Bay, South Burlington: {'primary': 'lake', 'alternate': ['structure_and_geography']}
  Executive Physical Therapy: {'primary': 'physical_therapy', 'alternate': ['occupational_therapy', 'speech_therapist']}
  Key Collision of South Burlington: {'primary': 'auto_body_shop', 'alternate': ['automotive_repair', 'automotive']}
  Shearer Chevrolet Buick GMC Cadillac: {'primary': 'automotive', 'alternate': None}
  W.B. Mason: {'primary': 'office_equipment', 'alternate': ['b2b_scientific_equipment', 'business_to_business', 'retail']}
  Burlington Self Storage: {'primary': 'self_storage_facility', 'alternate': ['storage_facility']}
  LCB Design Group: {'primary': 'interior_design', 'alternate': ['fashion', 'home_improvement_store']}
  Portrait Gallery: {'primary': 'professional_services', 'alternate': ['event_photog

In [7]:
# Query all restaurants in Burlington
restaurants_df = con.sql(f"""
SELECT 
    id,
    names.primary as name,
    categories.primary as primary_category,
    categories,
    ST_X(geometry) as lon,
    ST_Y(geometry) as lat,
    geometry
FROM read_parquet('s3://overturemaps-us-west-2/release/2026-01-21.0/theme=places/type=place/*.parquet')
WHERE 
    bbox.xmin <= {bbox['xmax']}
    AND bbox.xmax >= {bbox['xmin']}
    AND bbox.ymin <= {bbox['ymax']}
    AND bbox.ymax >= {bbox['ymin']}
    AND ST_Intersects(geometry, ST_GeomFromText('{wkt_geom}'))
    AND (
        lower(categories.primary) LIKE '%restaurant%'
        OR lower(categories.primary) LIKE '%food%'
        OR lower(categories.primary) LIKE '%dining%'
        OR lower(categories.primary) LIKE '%cafe%'
        OR lower(categories.primary) LIKE '%coffee%'
    )
""").df()

print(f"Found {len(restaurants_df)} restaurants/food places in Burlington")
restaurants_df['primary_category'].value_counts().head(15)

Found 108 restaurants/food places in Burlington


primary_category
pizza_restaurant                      13
restaurant                            12
american_restaurant                   10
coffee_shop                           10
fast_food_restaurant                   6
italian_restaurant                     5
food_beverage_service_distribution     4
chinese_restaurant                     4
burger_restaurant                      4
mexican_restaurant                     4
sushi_restaurant                       3
health_food_store                      2
gluten_free_restaurant                 2
bar_and_grill_restaurant               2
japanese_restaurant                    2
Name: count, dtype: int64

In [8]:
# Convert to GeoDataFrame
restaurants_gdf = gpd.GeoDataFrame(
    restaurants_df,
    geometry=gpd.points_from_xy(restaurants_df['lon'], restaurants_df['lat']),
    crs="EPSG:4326"
)
print(f"GeoDataFrame shape: {restaurants_gdf.shape}")
restaurants_gdf.head()

GeoDataFrame shape: (108, 7)


,id,name,primary_category,categories,lon,lat,geometry
0,8387da64-0a22-41ff-b4db-ad526ba350ec,Chittenden Cider Mill,food_beverage_service_distribution,{'primary': 'food_beverage_service_distributio...,-73.173296,44.423268,POINT (-73.1733 44.42327)
1,170a8ed0-2459-4cb2-97ad-72bdf7373ec8,Pauline's Cafe & Restaurant,restaurant,"{'primary': 'restaurant', 'alternate': ['ameri...",-73.211749,44.421540,POINT (-73.21175 44.42154)
2,8104b942-6918-436b-8122-8ae98ae40a5f,Bariatrix Nutrition Corp.,health_food_store,"{'primary': 'health_food_store', 'alternate': ...",-73.209291,44.422629,POINT (-73.20929 44.42263)
3,6f498dde-9d59-445b-adff-cb3867a35e47,Pulcinella's,italian_restaurant,"{'primary': 'italian_restaurant', 'alternate':...",-73.211287,44.424808,POINT (-73.21129 44.42481)
4,6ec5d62e-9fc0-4025-a96f-950e7d737242,Lake-View House Restaurant,american_restaurant,"{'primary': 'american_restaurant', 'alternate'...",-73.210907,44.425017,POINT (-73.21091 44.42502)


## 2. Download Burlington Street Network with OSMnx

In [9]:
# Download walkable street network for Burlington
# OSMnx 2.0 uses bbox tuple: (west, south, east, north) = (min_lon, min_lat, max_lon, max_lat)
# Add small buffer for edge POIs

buffer = 0.02
osm_bbox = (
    bbox['xmin'] - buffer,  # west
    bbox['ymin'] - buffer,  # south  
    bbox['xmax'] + buffer,  # east
    bbox['ymax'] + buffer   # north
)

print(f"Downloading Burlington street network for bbox: {osm_bbox}")
G = ox.graph_from_bbox(osm_bbox, network_type='walk')
print(f"Network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

Network: 17260 nodes, 46370 edges


In [ ]:
# Add travel times for walking
# Graph already has edge lengths from OSMnx - DON'T recalculate after projecting!
# Walking speed: 4.5 km/h = 75 m/min

G_proj = ox.project_graph(G)

for u, v, data in G_proj.edges(data=True):
    length_m = data.get('length', 0)
    data['travel_time'] = length_m / 75.0  # minutes

# Verify
sample_edge = list(G_proj.edges(data=True))[0]
print(f"Sample edge length: {sample_edge[2]['length']:.1f}m, travel_time: {sample_edge[2]['travel_time']:.2f}min")

## 3. Generate Isochrones

Using OSMnx's ego_graph to find all nodes within X minutes walk, then create a polygon.

In [ ]:
def generate_isochrone(G, center_point, trip_time_minutes, crs_proj):
    """
    Generate an isochrone polygon for a given point and travel time.
    
    Args:
        G: Projected NetworkX graph with travel_time edge attribute
        center_point: (lon, lat) tuple in WGS84
        trip_time_minutes: Maximum travel time in minutes
        crs_proj: CRS of the projected graph
    
    Returns:
        Shapely Polygon in WGS84
    """
    # Find nearest node to the center point
    # Note: ox.nearest_nodes expects (X, Y) which is (lon, lat) for unprojected
    # But our graph is projected, so we need to project the point first
    center_gdf = gpd.GeoDataFrame(
        geometry=[Point(center_point)],
        crs="EPSG:4326"
    ).to_crs(crs_proj)
    center_proj = (center_gdf.geometry.iloc[0].x, center_gdf.geometry.iloc[0].y)
    
    nearest_node = ox.nearest_nodes(G, center_proj[0], center_proj[1])
    
    # Get subgraph of all nodes within trip_time_minutes
    subgraph = nx.ego_graph(G, nearest_node, radius=trip_time_minutes, distance='travel_time')
    
    if len(subgraph.nodes()) < 3:
        # Not enough nodes for a polygon, return a small buffer around the point
        return center_gdf.buffer(50).to_crs("EPSG:4326").geometry.iloc[0]
    
    # Get node coordinates
    node_points = [Point(G.nodes[node]['x'], G.nodes[node]['y']) for node in subgraph.nodes()]
    
    # Create convex hull (simple approach) or concave hull for better shape
    # Using convex hull for speed, can switch to alpha shape later
    points_gdf = gpd.GeoDataFrame(geometry=node_points, crs=crs_proj)
    isochrone = points_gdf.union_all().convex_hull
    
    # Convert back to WGS84
    isochrone_gdf = gpd.GeoDataFrame(geometry=[isochrone], crs=crs_proj)
    isochrone_wgs84 = isochrone_gdf.to_crs("EPSG:4326").geometry.iloc[0]
    
    return isochrone_wgs84

In [12]:
# Test with one POI
test_poi = restaurants_gdf.iloc[0]
print(f"Testing isochrone for: {test_poi['name']}")
print(f"Location: ({test_poi['lon']}, {test_poi['lat']})")

# Get the projected CRS from our graph
crs_proj = G_proj.graph['crs']

# Generate 5-minute isochrone
iso_5min = generate_isochrone(G_proj, (test_poi['lon'], test_poi['lat']), 5, crs_proj)
print(f"5-min isochrone area: {iso_5min.area:.6f} sq degrees")
print(f"Isochrone type: {type(iso_5min)}")

Testing isochrone for: Chittenden Cider Mill
Location: (-73.173296, 44.423268)
5-min isochrone area: 0.000001 sq degrees
Isochrone type: <class 'shapely.geometry.polygon.Polygon'>


## 4. Batch Generate Isochrones for All POIs

Generate 5, 10, and 15 minute walking isochrones for each restaurant.

In [14]:
from tqdm.auto import tqdm

TRIP_TIMES = [5, 10, 15]  # minutes

isochrone_records = []

for idx, poi in tqdm(restaurants_gdf.iterrows(), total=len(restaurants_gdf), desc="Generating isochrones"):
    for trip_time in TRIP_TIMES:
        try:
            iso = generate_isochrone(G_proj, (poi['lon'], poi['lat']), trip_time, crs_proj)
            isochrone_records.append({
                'poi_id': poi['id'],
                'poi_name': poi['name'],
                'minutes': trip_time,
                'geometry': iso
            })
        except Exception as e:
            print(f"Error for {poi['name']} at {trip_time}min: {e}")

print(f"\nGenerated {len(isochrone_records)} isochrones")

Generating isochrones:   0%|          | 0/108 [00:00<?, ?it/s]


Generated 324 isochrones


In [15]:
# Create isochrones GeoDataFrame
isochrones_gdf = gpd.GeoDataFrame(isochrone_records, crs="EPSG:4326")
print(f"Isochrones shape: {isochrones_gdf.shape}")
isochrones_gdf.head()

Isochrones shape: (324, 4)


,poi_id,poi_name,minutes,geometry
0,8387da64-0a22-41ff-b4db-ad526ba350ec,Chittenden Cider Mill,5,"POLYGON ((-73.17267 44.42326, -73.17267 44.423..."
1,8387da64-0a22-41ff-b4db-ad526ba350ec,Chittenden Cider Mill,10,"POLYGON ((-73.17267 44.42326, -73.17267 44.423..."
2,8387da64-0a22-41ff-b4db-ad526ba350ec,Chittenden Cider Mill,15,"POLYGON ((-73.17267 44.42326, -73.17267 44.423..."
3,170a8ed0-2459-4cb2-97ad-72bdf7373ec8,Pauline's Cafe & Restaurant,5,"POLYGON ((-73.21112 44.42153, -73.21113 44.421..."
4,170a8ed0-2459-4cb2-97ad-72bdf7373ec8,Pauline's Cafe & Restaurant,10,"POLYGON ((-73.21112 44.42153, -73.21113 44.421..."


In [ ]:
# Visualize with Lonboard
from lonboard import viz

viz(isochrones_gdf)